## Artistic Model Training

#### NOTES:  
* This is "NoGAN" based training, described in the DeOldify readme.
* This model prioritizes colorful renderings.  It has higher variation in renderings at different resolutions compared to the "stable" model

In [ ]:
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

In [ ]:
import os
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from deoldify.generators import *
from deoldify.critics import *
from deoldify.dataset import *
from deoldify.loss import *
from deoldify.save import *
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageFile

## Setup

In [3]:
path = Path('data')
path_hr = path/'images'
path_lr = path/'grayscaled'

proj_id = 'Finetuned'

gen_name = proj_id + '_gen'
pre_gen_name = gen_name + '_0'
crit_name = proj_id + '_crit'

name_gen = proj_id + '_image_gen'
path_gen = path/name_gen

TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

nf_factor = 1.5
pct_start = 1e-8

In [ ]:
#Debug
print(path_gen)

In [4]:
def get_data(bs:int, sz:int, keep_pct:float):
    return get_colorize_data(sz=sz, bs=bs, crappy_path=path_lr, good_path=path_hr, 
                             random_seed=None, keep_pct=keep_pct)

#def get_crit_data(classes, bs, sz):
def get_crit_data(classes, bs, sz):
    src = ImageList.from_folder(path, include=classes, recurse=True).split_by_rand_pct(0.1, seed=42)
    ll = src.label_from_folder(classes=classes)
    data = (ll.transform(get_transforms(max_zoom=2.), size=sz)
           .databunch(bs=bs).normalize(imagenet_stats))
    return data

def create_training_images(fn,i):
    dest = path_lr/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn).convert('LA').convert('RGB')
    img.save(dest)  
    
def save_preds(dl):
    i=0
    names = dl.dataset.items
    
    for b in dl:
        preds = learn_gen.pred_batch(batch=b, reconstruct=True)
        for o in preds:
            o.save(path_gen/names[i].name)
            i += 1
    
def save_gen_images():
    if path_gen.exists(): shutil.rmtree(path_gen)
    path_gen.mkdir(exist_ok=True)
    data_gen = get_data(bs=bs, sz=sz, keep_pct=0.085)
    save_preds(data_gen.fix_dl)
    PIL.Image.open(path_gen.ls()[0])

## Create black and white training images

Only runs if the directory isn't already created.

In [10]:
if not path_lr.exists():
    il = ImageList.from_folder(path_hr)
    parallel(create_training_images, il.items)

/home/edgelab/.local/lib/python3.9/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (121403772 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


## Pre-train generator

#### NOTE
Most of the training takes place here in pretraining for NoGAN.  The goal here is to take the generator as far as possible with conventional training, as that is much easier to control and obtain glitch-free results compared to GAN training.

### 64px

In [5]:
bs=88
sz=64
keep_pct=1.0

In [ ]:
data_gen = get_data(bs=bs, sz=sz, keep_pct=keep_pct)

In [ ]:
#learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor)

In [ ]:
#learn_gen.callback_fns.append(partial(ImageGenTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GenPre'))

In [ ]:
#learn_gen.fit_one_cycle(1, pct_start=0.8, max_lr=slice(1e-3))

In [ ]:
#learn_gen.save(pre_gen_name)

In [ ]:
#learn_gen.unfreeze()

In [ ]:
#learn_gen.fit_one_cycle(1, pct_start=pct_start,  max_lr=slice(3e-7, 3e-4))

In [ ]:
#learn_gen.save(pre_gen_name)

### 128px

In [ ]:
bs=22
sz=128
keep_pct=1.0

In [ ]:
#learn_gen.data = get_data(sz=sz, bs=bs, keep_pct=keep_pct)

In [ ]:
#learn_gen.unfreeze()

In [ ]:
#learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(1e-7,1e-4))

In [ ]:
#learn_gen.save(pre_gen_name)

### 192px

In [ ]:
#bs = 11
#bs=9
#sz=192
#keep_pct=0.50

In [ ]:
#learn_gen.data = get_data(sz=sz, bs=bs, keep_pct=keep_pct)

In [ ]:
#learn_gen.unfreeze()

In [ ]:
#learn_gen.fit_one_cycle(1, pct_start=pct_start, max_lr=slice(5e-8,5e-5))

In [ ]:
#print(pre_gen_name)

In [ ]:
#learn_gen.save(pre_gen_name)

## Repeatable GAN Cycle

#### NOTE
Best results so far have been based on repeating the cycle below a few times (about 5-8?), until diminishing returns are hit (no improvement in image quality).  Each time you repeat the cycle, you want to increment that old_checkpoint_num by 1 so that new check points don't overwrite the old.  

In [7]:
old_checkpoint_num = 4
checkpoint_num = old_checkpoint_num + 1
gen_old_checkpoint_name = gen_name + '_' + str(old_checkpoint_num)
gen_new_checkpoint_name = gen_name + '_' + str(checkpoint_num)
crit_old_checkpoint_name = crit_name + '_' + str(old_checkpoint_num)
crit_new_checkpoint_name= crit_name + '_' + str(checkpoint_num)

### Save Generated Images

In [8]:
# batch size and sample size...
# 18 images
# No size of images matter
# OG:
#bs=8
#sz=192
bs=8
sz=192
keep_pct=1.0

In [ ]:
print(gen_old_checkpoint_name)
print(path)
print(bs)
print(sz)
print(keep_pct)

In [ ]:
data_gen = get_data(bs=bs, sz=sz, keep_pct=keep_pct)

In [ ]:
#print(data_gen)

In [ ]:
#data_gen.show_batch(rows=4, ds_type=DatasetType.Train, imgsize=3)

In [ ]:
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor).load(gen_old_checkpoint_name, with_opt=False)

In [ ]:
save_gen_images()

### Pretrain Critic

##### Only need full pretraining of critic when starting from scratch.  Otherwise, just finetune!

In [ ]:
print(name_gen)

In [ ]:
"""
if old_checkpoint_num == 0:
    bs=64
    sz=128
    learn_gen=None
    gc.collect()
    data_crit = get_crit_data([name_gen, 'images'], bs=bs, sz=sz)
    data_crit.show_batch(rows=3, ds_type=DatasetType.Train, imgsize=3)
    learn_critic = colorize_crit_learner(data=data_crit, nf=256)
    learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))
    learn_critic.fit_one_cycle(6, 1e-3)
    learn_critic.save(crit_old_checkpoint_name)

"""

In [14]:
bs=16
sz=192

In [ ]:
data_crit = get_crit_data([name_gen, 'images'], bs=bs, sz=sz)

In [ ]:
# OG: rows=3 and imgsize=3
data_crit.show_batch(rows=5, ds_type=DatasetType.Train, imgsize=4)

In [ ]:
learn_critic = colorize_crit_learner(data=data_crit, nf=256).load(crit_old_checkpoint_name, with_opt=False)

In [18]:
learn_critic.callback_fns.append(partial(LearnerTensorboardWriter, base_dir=TENSORBOARD_PATH, name='CriticPre'))

In [ ]:
learn_critic.fit_one_cycle(4, 1e-4)

In [20]:
learn_critic.save(crit_new_checkpoint_name)

In [ ]:
print(crit_new_checkpoint_name)

### GAN

https://docs.fast.ai/vision.gan.html For fastai GAN docs
https://juliawu.medium.com/fast-ai-lesson-7of-7-resnet-unet-gans-245960585fd2 For tutorial that deoldify is partially based on

In [ ]:
learn_crit=None
learn_gen=None
gc.collect()

In [23]:
#lr=1e-5
lr=0.1
sz=192
bs=9

In [ ]:
data_crit = get_crit_data([name_gen, 'images'], bs=bs, sz=sz)

In [ ]:
learn_crit = colorize_crit_learner(data=data_crit, nf=256).load(crit_new_checkpoint_name, with_opt=False)

In [ ]:
learn_gen = gen_learner_deep(data=data_gen, gen_loss=FeatureLoss(), nf_factor=nf_factor).load(gen_old_checkpoint_name, with_opt=False)

In [27]:
switcher = partial(AdaptiveGANSwitcher, critic_thresh=0.65)
learn = GANLearner.from_learners(learn_gen, learn_crit, weights_gen=(1.0,2.0), show_img=False, switcher=switcher,
                                 opt_func=partial(optim.Adam, betas=(0.,0.9)), wd=1e-3)
learn.callback_fns.append(partial(GANDiscriminativeLR, mult_lr=5.))
learn.callback_fns.append(partial(GANTensorboardWriter, base_dir=TENSORBOARD_PATH, name='GanLearner', visual_iters=100))
learn.callback_fns.append(partial(GANSaveCallback, learn_gen=learn_gen, filename=gen_new_checkpoint_name, save_iters=100))

#### Instructions:  
This is all very new so you may need to play around with just how far you go here with keep_pct.

In [ ]:
#learn.data = get_data(sz=sz, bs=bs, keep_pct=0.03)
learn.data = get_data(sz=sz, bs=bs, keep_pct=1)
learn_gen.freeze_to(-1)
learn.fit(1,lr)

In [29]:
learn_gen.save(gen_new_checkpoint_name)

In [ ]:
import filecmp
#Check that the weights have been updated, False=Updated
filecmp.cmp('data/grayscaled/models/Finetuned_gen_0.pth', 'data/grayscaled/models/Finetuned_gen_1.pth')
